# Model nad podacima iz Srbije

Sada ćemo naš model za klasifikaciju namene zemljišta primeniti na podacima iz Srbije. Kao i za bazu na kojoj je model treniran, koristićemo Sentinel 2 podatke ali regije koja odgovara Srbiji. U ovom primeru smo izabrali Sentinel 2 snimke šire okoline Šapca od 30. juna 2021. godine.

In [ ]:
from os import environ
environ["OPENCV_IO_ENABLE_JASPER"] = "true"
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision

import cv2

import numpy as np

from skimage import exposure
from sklearn import metrics

from matplotlib import pyplot as plt

U ćeliji ispod učitavamo arhitekturu modela i prilagođavamo je da predviđa 10 klasa (u zadnjoj liniji). Takođe, torch.no_grad(): naglašava da se ne vrši nikakvo treniranje nad modelom unutar njega. Pod nadovnicima je potrebno da navedete putanju do sačuvanog modela na vašem računaru.

In [ ]:
with torch.no_grad():
    model = torchvision.models.resnet50(pretrained=True)
    num_features = model.fc.in_features
    model.fc = nn.Linear(num_features, 10)

device = "cpu"
model.to(device)

model.load_state_dict(torch.load(r"", map_location=torch.device(device)))
model.eval()
print("Model ucitan")

Sami podaci koji predstavljaju Sentinel 2 proizvode kada se raspakuju sadrže veliki broj snimaka i metapodataka. U ovoj vežbanci ćemo se prevashodno fokusirati na slike u vidljivom spektru boja (praktično standardne slike kakve svakodnevno gledamo). Sam satelit ima instrument koji snima u više spektralnih opsega (eng. spectral bands), a nama su od interesa oni koji snimaju na talasnim dužinama crvene zelene i plave boje koji su dovoljni za formiranje slike u boji. U pitanju su spektralni opsezi sa indeksima b02, b03 i b04.

In [ ]:
img_path_b02 = r"datasets/S2A_MSIL2A/T34TCQ_20210630T093041_B02_10m.jp2"
img_path_b03 = r"datasets/S2A_MSIL2A/T34TCQ_20210630T093041_B03_10m.jp2"
img_path_b04 = r"datasets/S2A_MSIL2A/T34TCQ_20210630T093041_B04_10m.jp2"

Funkcija ispod učitava svaki od snimaka za pojedinačnu boju i onda ih kombinuje u jednu sliku. Kada je pozovemo nad putanjama za svaku od slika imaćemo učitan celokupan snimak u boji. Slike su jako velike, pa ovaj korak može potrajati koji minut.

In [ ]:
def load_sentinel_2_image_rgb(img_path_b02, img_path_b03, img_path_b04):
    img_b02 = cv2.imread(img_path_b02, cv2.IMREAD_ANYCOLOR | cv2.IMREAD_ANYDEPTH)
    img_b03 = cv2.imread(img_path_b03, cv2.IMREAD_ANYCOLOR | cv2.IMREAD_ANYDEPTH)
    img_b04 = cv2.imread(img_path_b04, cv2.IMREAD_ANYCOLOR | cv2.IMREAD_ANYDEPTH)

    rgb = np.zeros((img_b02.shape[0], img_b02.shape[1], 3))
    rgb[:, :, 0] = img_b04
    rgb[:, :, 1] = img_b03
    rgb[:, :, 2] = img_b02

    rgb = rgb.astype(np.uint16)

    return rgb

In [ ]:
image = load_sentinel_2_image_rgb(#DOPUNI)

Potrebno je da slikama malo promeniti kontrast i osvetljenost tako da bolje odgovaraju observacijama za Srbiju. To je poenta funkcije ispod. Sami parametri modifikacije su ručno izabrani posmatrajući oba domena slika, mada je potpuno moguće i poželjno u praksi ovo uraditi na sistematičan način. Takođe u zavisnosti od primene, potrebno je primeniti i složenije tehnike domenske adaptacije.

In [ ]:
def prepare_rgb(image):
    p_low, p_high = np.percentile(image, (1, 99))
    image = exposure.rescale_intensity(image, in_range=(p_low, p_high))
    image = image / image.max()
    
    image = exposure.adjust_gamma(image, 0.9)
    image += 0.05
    image = np.clip(image, 0.0, 1.0)

    return image

In [ ]:
# Шабац - севернији део и река Сава
CROP_START_COL = 9584
CROP_START_ROW = 4060
CROP_SIZE = 512

# Шабац - јужнији део
# CROP_START_COL = 9423
# CROP_START_ROW = 4329
# CROP_SIZE = 512

# Рума - јужни део
# CROP_START_COL = 10383
# CROP_START_ROW = 1626
# CROP_SIZE = 512

# Јарак - цело место и меандар Саве
# CROP_START_COL = 9929
# CROP_START_ROW = 2348
# CROP_SIZE = 512

# Вештачко језеро Ровни
# CROP_START_COL = 9721
# CROP_START_ROW = 9908
# CROP_SIZE = 512

sample_image_1 = image[CROP_START_ROW:CROP_START_ROW+CROP_SIZE, CROP_START_COL:CROP_START_COL+CROP_SIZE, :]

U ćeliji ispod prikažite sliku

In [ ]:
#DOPUNI

Naš model mašinskog učenja kao ulaz uzima slike veličine 64x64, dok je naš isečak veličine 512x512. Da bismo obradili ceo isečak moramo ga podeliti u manje isečke veličine 64x64. Takvih isečaka će biti 8 po dužini i 8 po širini, što znači ukupno 64. Tih 64 manjih isečaka će činiti hrpu koju ćemo kao tavku dati mreži i od nje dobiti odgovarajući broj predikcija.

Funkcije ispod se bave upravo ovom pripremom. 
Konkretno, funkcija get_images dati isečak deli u skup malih pločica veličine 64x64, a funkcija prepare_images_for_dnn pakuje tu hrpu pločica u strukturu padataka adekvatne veličine koju veštačka neuralna mreža može da prihvati.

In [ ]:
def get_images(image):
    image = prepare_rgb(image)

    rows = #DOPUNI
    cols = #DOPUNI
    all_tiles = np.zeros((rows * cols, 64, 64, 3), dtype=np.float32)

    for r in range(rows):
        for c in range(cols):
            all_tiles[r + rows * c, ...] = image[r*64:r*64+64, c*64:c*64+64, :]
    
    return all_tiles

def prepare_images_for_dnn(images):
    images_prepared = torch.zeros((images.shape[0], 3, 224, 224))

    for i in range(images.shape[0]):

            image_resized = cv2.resize(images[i, ...], (224, 224))
            img_normalized = image_resized.transpose(2, 0, 1).astype(np.float32)

            images_prepared[i, ...] = torch.from_numpy(img_normalized)

    return images_prepared

In [ ]:
all_tiles = get_images(image[CROP_START_ROW:CROP_START_ROW+CROP_SIZE, CROP_START_COL:CROP_START_COL+CROP_SIZE, :])
tiles_prepared_for_dnn = prepare_images_for_dnn(#DOPUNI).to(device)

Kao i u prethodnim vežbama potrebno je da model napravi neke predikcije ulaznih podataka

In [ ]:
with torch.no_grad():
    pred = #DOPUNI

Pronađite i štampajte redni broj maksimalnog elementa:

In [ ]:
#DOPUNI

S obzirom da model predviđa skor za svaku od klasa, je moguće dati i estimaciju verovatnoće sa kojom je model siguran u svoju predikciju. U mašinskom učenju se za te potrebe najčešće koristi sofmax funkcija u koju nećemo dalje zalaziti, ali je dovoljno zapamtiti da ona vektor skorova pretvara u aproksimativnu raspodelu verovatnoće.

In [ ]:
with torch.no_grad():
    softmaxes = #DOPUNI

In [ ]:
all_scores = np.take_along_axis(softmaxes.numpy().T, np.expand_dims(argmax.numpy(), 0), 0)
print(all_scores)

Srednja vrednost:

In [ ]:
#DOPUNI